Для запуска нужно поставить:

- MXNet. See the instructions for your operating system in [Setup and Installation](http://mxnet.io/install/index.html)

- [Python Requests](http://docs.python-requests.org/en/master/), [Matplotlib](https://matplotlib.org/) and [Jupyter Notebook](http://jupyter.org/index.html).

Также потребуются numpy, cv2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
import mxnet as mx
import os
import glob
import time
from collections import namedtuple

Далее описаны две функции. Первая помогает выкачивать изображение и обрабатывать его, вторая собственно предсказывает результат модели на изображении. Она сохраняет 5 наиболее вероятных классов, к которым может относиться изображение, и вероятности этих классов.

In [2]:
#Код из примера, выкачивает картинку и предсказывает классы

Batch = namedtuple('Batch', ['data'])

def get_image(url, show=False, local=False):
    # download and show the image
    if (local == False):
        fname = mx.test_utils.download(url, dirname='images')
    else:
        fname = url
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    if img is None:
         return None
    if show:
         plt.imshow(img)
         plt.axis('off')
    # convert into format (batch, RGB, width, height)
    img = cv2.resize(img, (224, 224))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
    return img

def predict_old(url, show=False, local=False):
    res = []
    img = get_image(url, show=show, local=local)
    # compute the predict probabilities
    #print(img)
    mod.forward(Batch([mx.nd.array(img)]))
    #print('what we give to forward')
    #print([mx.nd.array(img)])
    prob = mod.get_outputs()[0].asnumpy()
    # print the top-5
    prob = np.squeeze(prob)
    a = np.argsort(prob)[::-1]
    for i in a[0:5]:
        res.append([prob[i], labels[i]])
        #print('probability=%f, class=%s' %(prob[i], labels[i]))
    return res

In [14]:
predict_old('https://avatars.mds.yandex.net/get-pdb/906476/9ed1624b-804d-4d26-b9d5-2eac53b4fb03/s800')

[[0.96191114, 'n04330267 Stove'],
 [0.02427841, 'n04574999 Wheel'],
 [0.0050751548, 'n02445004 River otter, Lutra canadensis'],
 [0.0027608434, 'n04289027 Sprinkler'],
 [0.00096800085, 'n02367492 Chinchilla, Chinchilla laniger']]

Я выкачал 4 предобученные модели и протестировал их на руками собранных выборках картинок ручек, ключей, телефонов. Результаты:

In [3]:
models = ['Inception', 'caffenet', 'nin', 'squeezenet_v1.0', 'vgg19']#, 'vgg16']

for i, name in enumerate(models):
    #Initializing model
    print('Model ' + name)
    sym, arg_params, aux_params = mx.model.load_checkpoint(name + '/' + name, 0)
    mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
    mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
             label_shapes=mod._label_shapes)
    mod.set_params(arg_params, aux_params, allow_missing=True)
    with open(name + '/synset.txt', 'r') as f:
        labels = [l.rstrip() for l in f]
    #Predict testing
    datasets = ['pen', 'key', 'phone']
    sz = 0
    start = time.time()
    for i, dataset in enumerate(datasets):
        ans = np.zeros(5)
        cnt = np.zeros(6)
        for filename in glob.iglob('images/' + dataset + '/*'):
            result = predict_old(filename,  local=True)
            was = False
            for tmp in result:
                prob = tmp[0]
                tag = tmp[1].lower()
                if (tag.find(dataset) != -1):
                    ans[i] += prob
                    cnt[i] += 1
                    was = True
                    break
            if (not was):
                cnt[5] += 1
        print('For dataset ' + dataset + ':')
        for i in range(5):
            if (cnt[i] > 0):
                print('Recognised with ' + str(i + 1) + ' priority: ' + str(cnt[i]) + 
                  ' images with average probability =  ' + str(ans[i] / cnt[i]))
            else:
                print('Recognised with ' + str(i + 1) + ' priority: ' + str(cnt[i]))
        print('Not recognized: ', cnt[5])
        sz += np.sum(cnt)
    end = time.time()
    print(str(sz) + ' images were analyzed in ' + str(end - start) + ' seconds.')
    print

Model Inception
For dataset pen:
Recognised with 1 priority: 23.0 images with average probability =  0.223848933071
Recognised with 2 priority: 0.0
Recognised with 3 priority: 0.0
Recognised with 4 priority: 0.0
Recognised with 5 priority: 0.0
('Not recognized: ', 9.0)
For dataset key:
Recognised with 1 priority: 0.0
Recognised with 2 priority: 17.0 images with average probability =  0.236642334282
Recognised with 3 priority: 0.0
Recognised with 4 priority: 0.0
Recognised with 5 priority: 0.0
('Not recognized: ', 19.0)
For dataset phone:
Recognised with 1 priority: 0.0
Recognised with 2 priority: 0.0
Recognised with 3 priority: 5.0 images with average probability =  0.140878477693
Recognised with 4 priority: 0.0
Recognised with 5 priority: 0.0
('Not recognized: ', 27.0)
100.0 images were analyzed in 87.9958999157 seconds.

Model caffenet
For dataset pen:
Recognised with 1 priority: 26.0 images with average probability =  0.294231741259
Recognised with 2 priority: 0.0
Recognised with 3 

In [20]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
import mxnet as mx
import os
import glob
import time
from collections import namedtuple

class ModelServer:
    #Я считаю, что в config передается пара ('путь к папке с моделью'; число классов, которые надо возвращать)
    #При этом название папки с моделью совпадает с именем модели!!!
    def __init__(self, config):
        #Код далее подгружает модель
        #В папке должны лежать файлы .params и .json с параметрами, а также synset.txt со списком классов
        path = config[0]
        name = os.path.split(path)[1]
        self.sym, self.arg_params, self.aux_params = mx.model.load_checkpoint(path + '/' + name, 0)
        self.mod = mx.mod.Module(symbol=self.sym, context=mx.cpu(), label_names=None)
        self.mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
                 label_shapes=self.mod._label_shapes)
        self.mod.set_params(self.arg_params, self.aux_params, allow_missing=True)
        with open(path + '/synset.txt', 'r') as f:
            self.labels = [l.rstrip() for l in f]
        self.n = config[1]

    def run_single(self, img):
        #Мы решили, что картинка передается как матрица (ширина, высота, число каналов)
        #Сначала я преобразую ее в нужный формат
        #Картинка может быть любого размера, я преобразую ее в 224х224 (так было в примере кода, я решил оставить)
        img = np.swapaxes(img, 0, 1)
        img = cv2.resize(img, (224, 224))
        img = np.swapaxes(img, 0, 2)
        img = np.swapaxes(img, 1, 2)
        img = img[np.newaxis, :]
        #Здесь нашей модели скрамливаем картинку и смотрим, что она ответит
        Batch = namedtuple('Batch', ['data'])
        self.mod.forward(Batch([mx.nd.array(img)]))
        prob = self.mod.get_outputs()[0].asnumpy()
        prob = np.squeeze(prob)
        #Вернем n классов, к которым мы принадлежим с наибольшей вероятностью
        predictions = np.argsort(prob)[::-1]
        res = []
        for i in predictions[0:self.n]:
            res.append([prob[i], self.labels[i]])
        return res

    #Здесь принимается массив картинок в таком же формате, как и в run_single
    #Для каждой просто вызывается run_single
    def run_chunk(self, imgs):
        ans = []
        for img in imgs:
            ans.append(self.run_single(img))
        return ans

    #def run_stream(input_stream<tensor<w,h,c>>, output_stream<label>):


In [21]:
ttmp = ['Inception', 'caffenet', 'nin', 'squeezenet_v1.0', 'vgg19']

#model = ModelServer('vgg19')
del model
model = ModelServer(['/home/alladdin/inprak/vgg19', 5])

In [22]:
#img = cv2.cvtColor(cv2.imread('images/1434596941_632146314.jpg'), cv2.COLOR_BGR2RGB)
img = cv2.cvtColor(cv2.imread('images/1.jpeg'), cv2.COLOR_BGR2RGB)
img = np.swapaxes(img, 0, 1)
img2 = cv2.cvtColor(cv2.imread('images/123.jpg'), cv2.COLOR_BGR2RGB)
img2 = np.swapaxes(img2, 0, 1)
model.run_chunk(np.array([img, img2]))
del model